In [1]:
import os
import cv2
import time
import numpy as np
import pandas as pd
import mediapipe as mp
from matplotlib import pyplot as plt

In [2]:
df = pd.read_excel("SignTlak Dataset - Copy.xlsx")
df

,VIDEO_ID,SIGNER_ID,IS_SENTENCE,SIGN
0,MVI_3949,1,0,Health
1,MVI_3950,1,0,Human
2,MVI_3951,1,0,Head
3,MVI_3952,1,0,Mind
4,MVI_3953,1,0,Brain
...,...,...,...,...
1066,MVI_5264,1,1,"Yes, make sure you're getting plenty of rest a..."
1067,MVI_5266,1,1,Try to eat a balanced diet and get some light ...
1068,MVI_5267,1,1,Try to eat a balanced diet and get some light ...
1069,MVI_5268,1,1,"Okay, I'll do my best. Thanks, doctor."


In [3]:
# Grabbing the Holistic Model from Mediapipe and
# Initializing the Model

mp_holistic = mp.solutions.holistic
holistic_model = mp_holistic.Holistic(
	min_detection_confidence=0.5,
	min_tracking_confidence=0.5
)

# Initializing the drawing utils for drawing the facial landmarks on image
mp_drawing = mp.solutions.drawing_utils


In [4]:
X = []
y = []

In [5]:
video_path = "G:/DCIM/100CANON/"
video_folder = os.listdir(video_path)
for index, row in df.iterrows():
    video_id, sign = row['VIDEO_ID'], row['SIGN']
    video = video_id + ".MOV"

    if video in video_folder:

        curr_landmarks = []       

        video_dir = video_path + video
        capture = cv2.VideoCapture(video_dir)

        # Initializing current time and precious time for calculating the FPS
        previousTime = 0
        currentTime = 0

        while capture.isOpened():
	        # capture frame by frame
            ret, frame = capture.read()
            if not ret:
                break

	        # resizing the frame for better view
            frame = cv2.resize(frame, (800, 600))
            
            # Converting the from BGR to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

	        # Making predictions using holistic model
	        # To improve performance, optionally mark the image as not writeable to
	        # pass by reference.
            
            image.flags.writeable = False
            results = holistic_model.process(image)
            image.flags.writeable = True

	        # Converting back the RGB image to BGR
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
			# Process pose
            pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
            face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
            lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
            rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
            frame_pose = np.concatenate([pose, face, lh, rh])
            curr_landmarks.append(frame_pose)
            
			

	        # Drawing the Facial Landmarks
            mp_drawing.draw_landmarks(
	            image,
	            results.face_landmarks,
	            mp_holistic.FACEMESH_CONTOURS,
	            mp_drawing.DrawingSpec(
		        color=(255,0,255),
		        thickness=1,
		        circle_radius=1
	        ),
	
            mp_drawing.DrawingSpec(
		        color=(0,255,255),
		        thickness=1,
		        circle_radius=1
	        )
	        )

	        # Drawing Right hand Land Marks
            mp_drawing.draw_landmarks(
	            image,
	            results.right_hand_landmarks,
	            mp_holistic.HAND_CONNECTIONS
	        )

	        # Drawing Left hand Land Marks
            mp_drawing.draw_landmarks(
	            image,
	            results.left_hand_landmarks,
	            mp_holistic.HAND_CONNECTIONS
	        )

	        # Drawing pose Land Marks
            mp_drawing.draw_landmarks(
	            image,
	            results.pose_landmarks,
	            mp_holistic.POSE_CONNECTIONS
	        )
	
	        # Calculating the FPS
            currentTime = time.time()
            fps = 1 / (currentTime-previousTime)
            previousTime = currentTime
	
	        # Displaying FPS on the image
            cv2.putText(image, str(int(fps))+" FPS", (10, 70), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)

	        # Display the resulting image
            cv2.imshow("Facial and Hand Landmarks", image)

	        # Enter key 'q' to break the loop
            if cv2.waitKey(5) & 0xFF == ord('q'):
                break

            # When all the process is done
            # Release the capture and destroy all windows
        
        y.append(sign)
        X.append(curr_landmarks)

        capture.release()
        cv2.destroyAllWindows()

        # print(X)
        # print(y)

In [6]:
print(len(X))
print(len(y))

1059
1059


In [7]:
X = np.array(X)

C:\Users\Sam\AppData\Local\Temp\ipykernel_9984\55319187.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array(X)


In [9]:
np.save("wed_X.npy", X)

In [10]:
np.save("wed_y.npy", y)

In [19]:
max_length = max(len(subarray) for subarray in X)
print(max_length)

372


In [1]:
import numpy as np
wed_X = np.load("wed_X.npy", allow_pickle=True)

In [2]:
max_length = max(len(subarray) for subarray in wed_X)
print(max_length)

372


In [3]:
len(wed_X[0][2])

1662

In [5]:
x= wed_X[:100]

In [6]:
x = np.asarray(x).astype('float32')

ValueError: setting an array element with a sequence.

In [19]:
# Step 2 and 3: Pad sublists with arrays of zeros to match the maximum length
for sublist in wed_X:
    
    while len(sublist) < max_length:
        sublist.append(np.array([0] * 1662))
    sublist = np.array(sublist)

In [23]:
new_X  = wed_X[:200]

In [7]:
import tensorflow as tf
from tensorflow.keras import layers, optimizers

In [8]:
type(wed_X[-2])

list

In [56]:
print(len(set(y)))

870


In [9]:
# include early stopping and reducelr
def get_callbacks():
    return [
            tf.keras.callbacks.EarlyStopping(
            monitor="val_accuracy",
            patience = 10,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor = "val_accuracy",
            factor = 0.5,
            patience = 3
        ),
    ]

# a single dense block followed by a normalization block and relu activation
def dense_block(units, name):
    fc = layers.Dense(units)
    norm = layers.LayerNormalization()
    act = layers.Activation("relu")
    drop = layers.Dropout(0.1)
    return lambda x: drop(act(norm(fc(x))))

# the lstm block with the final dense block for the classification
def classifier(lstm_units):
    lstm = layers.LSTM(lstm_units)
    out = layers.Dense(870, activation="softmax")
    return lambda x: out(lstm(x))
# choose the number of nodes per layer
encoder_units = [512, 256] # tune this
lstm_units = 250 # tune this

#define the inputs (ragged batches of time series of landmark coordinates)
inputs = tf.keras.Input(shape=((1059, 372)), ragged=True)

# dense encoder model
x = inputs
for i, n in enumerate(encoder_units):
    x = dense_block(n, f"encoder_{i}")(x)

# classifier model
out = classifier(lstm_units)(x)

model = tf.keras.Model(inputs=inputs, outputs=out)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1059, 372)]       0         
                                                                 
 dense (Dense)               (None, 1059, 512)         190976    
                                                                 
 layer_normalization (LayerN  (None, 1059, 512)        1024      
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 1059, 512)         0         
                                                                 
 dropout (Dropout)           (None, 1059, 512)         0         
                                                                 
 dense_1 (Dense)             (None, 1059, 256)         131328    
                                                             

In [10]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
y_encoded = encoder.fit_transform(np.array(y).reshape(-1, 1))

NameError: name 'y' is not defined

In [ ]:
# X_padded = tf.keras.preprocessing.sequence.pad_sequences(wed_X, padding='post')

# Convert X and y datasets to TensorFlow tensors
# X_dataset = tf.convert_to_tensor(X_padded, dtype=tf.float32)
# y_encoded = tf.convert_to_tensor(y_encoded, dtype=tf.float32)

In [15]:
X_train = wed_X.astype(np.uint8)

ValueError: setting an array element with a sequence.

In [12]:
x_train = np.array([np.array(val) for val in wed_X])
# y_train = np.array([np.array(val) for val in y_train])


MemoryError: Unable to allocate 4.88 GiB for an array with shape (1059, 372, 1662) and data type float64

In [13]:
x_train = tf.cast(wed_X , dtype=tf.float32)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

In [60]:
optimizer = optimizers.Adam()

model.compile(optimizer=optimizer,
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy","sparse_top_k_categorical_accuracy"])
# fit the model with 100 epochs iteration
model.fit(wed_X,
          y_encoded,
          callbacks = get_callbacks(),
          epochs = 10)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

In [121]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [122]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [178]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(1059, 372)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(870, activation='softmax'))

In [179]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [127]:
label_map = {label:num for num, label in enumerate(y)}
print(label_map)

{'Health': 0, 'Human': 1, 'Head': 639, 'Mind': 3, 'Brain': 4, 'Live': 5, 'Face': 640, 'Eye': 643, 'Ear': 8, 'Nose': 9, 'Mouth': 10, 'Teeth': 11, 'Beard': 12, 'Hand': 13, 'Feet': 14, 'Blood': 758, 'Breast': 726, 'Penis': 714, 'Anus': 711, 'Flatulence': 19, 'Spit': 20, 'Sweat': 21, 'Sleep': 22, 'Lie down': 790, 'Bath': 952, 'Smell': 25, 'Hear': 26, 'Blind': 27, 'Sick': 798, 'Infection': 664, 'Deaf': 661, 'Cut': 716, 'Sore': 32, 'Diarrhea': 723, 'Cold': 689, 'Convulsion': 36, 'Headache': 37, 'Chicken pox': 38, 'Malaria': 39, 'Tuberclosis': 40, 'Ebola': 41, 'Cancer': 42, 'HIV': 702, 'AIDS': 663, 'Transmit': 45, 'STI': 701, 'Mental': 47, 'Disabled': 48, 'Crippled': 49, 'Hospital': 803, 'Surgery ': 51, 'Sex': 667, 'gender': 53, 'Lesbian': 54, 'Semen': 56, 'Erection': 57, 'Condom': 669, 'Vaginal fluid': 59, 'Menstruation': 60, 'Pregnant': 717, 'Birth': 715, 'Breastfeed': 63, 'Female circumcision': 64, 'Female masturbation ': 65, 'Female masturbation': 66, 'Abortion': 67, 'Smoking': 68, 'Wee':

In [129]:
np.array(y).shape

(1059,)

In [130]:
X_train = np.array(padded_arr)

C:\Users\Sam\AppData\Local\Temp\ipykernel_508\2745492590.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train = np.array(padded_arr)


In [131]:
X_train.shape

(1059, 372)

In [196]:
len(X[6][1])

1662

In [198]:
import numpy as np
padded_arrays = []
def pad_2d_arrays(array_list):
    max_length = max(len(arr) for arr in array_list)  # Find the maximum length of 2D arrays

    
    for arr in array_list:
        pad_width = ((0, max_length - len(arr)), (0, 1662))  # Calculate the padding width
        padded_arr = np.pad(arr, pad_width, mode='constant')  # Pad the array
        padded_arrays.append(padded_arr)

    return padded_arrays

# Example usage
# Assuming array_list is your list containing the 2D arrays
padded_list = pad_2d_arrays(X)


ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (2,2)  and requested shape (1,2)

In [134]:
from sklearn.preprocessing import OneHotEncoder

# Create an instance of the OneHotEncoder
encoder = OneHotEncoder(sparse=False)

# Reshape the word list to a 2D array (n_samples, 1)
word_list_2d = [[word] for word in y]

# Perform one-hot encoding
one_hot_encoded = encoder.fit_transform(word_list_2d)

c:\Users\Sam\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [181]:
model.fit(np.array(X), y, epochs=2000, callbacks=[tb_callback])

C:\Users\Sam\AppData\Local\Temp\ipykernel_508\1045748640.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  model.fit(np.array(X), y, epochs=2000, callbacks=[tb_callback])


ValueError: Failed to find data adapter that can handle input: <class 'numpy.ndarray'>, (<class 'list'> containing values of types {"<class 'str'>"})

In [183]:
print(np.array(X).dtype)

object


C:\Users\Sam\AppData\Local\Temp\ipykernel_508\2318462743.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  print(np.array(X).dtype)


In [145]:
def check_nested_array(nested_array):
    # Check the structure of the nested array
    nested_array_length = len(nested_array[0])  # Assuming all nested arrays have the same length
    for nested in nested_array:
        if len(nested) != nested_array_length:
            print(len(nested))
            return False
    
    # Check the data types within the nested array
    expected_type = type(nested_array[0][0])  # Assuming all elements have the same type
    for nested in nested_array:
        for element in nested:
            print(type(element))
            if type(element) != expected_type:
                print(type(element))
                return False
    
    return True

# Example usage
is_valid = check_nested_array(padded_arr)
print(is_valid)  # Output: True


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.nd

In [174]:
padded_arr[-650]

[array([ 0.50286561,  0.24282664, -0.52643996, ...,  0.52174175,
         0.91284168, -0.04412461]),
 array([ 0.49220952,  0.26368633, -0.52138311, ...,  0.        ,
         0.        ,  0.        ]),
 array([ 0.48695838,  0.26897791, -0.50165528, ...,  0.        ,
         0.        ,  0.        ]),
 array([ 0.48648584,  0.26829833, -0.46982089, ...,  0.        ,
         0.        ,  0.        ]),
 array([ 0.48951045,  0.26551896, -0.46566802, ...,  0.51998889,
         0.75455689, -0.0337056 ]),
 array([ 0.49124673,  0.26485133, -0.45838943, ...,  0.52111924,
         0.72124308, -0.03975023]),
 array([ 0.49239612,  0.26692066, -0.42371807, ...,  0.52017283,
         0.69272232, -0.03785754]),
 array([ 0.49176252,  0.2659457 , -0.40783477, ...,  0.51705527,
         0.66946453, -0.03609408]),
 array([ 0.49232879,  0.26530331, -0.41583815, ...,  0.51378584,
         0.65177101, -0.03582725]),
 array([ 0.48999047,  0.26471007, -0.41455132, ...,  0.51432335,
         0.6380294 , -0.03

In [177]:
import numpy as np

def count_int_subarrays(nested_array):
    count = 0
    for subarray in nested_array:
        if any(isinstance(element, int) for element in subarray):
            count += 1
    return count

# Example usage
nested_array = padded_arr
int_subarray_count = count_int_subarrays(nested_array)
print(int_subarray_count)  # Output: 2


1058


In [188]:
padded_arr[0][2]

array([ 0.4571802 ,  0.22804336, -0.527282  , ...,  0.        ,
        0.        ,  0.        ])

In [186]:
for i in padded_arr:
    for y in 
    if len(i) != 372:
        print(False)
        break

In [208]:
X_processed = []
for sublist in X:
    if isinstance(sublist[0], list):
        X_processed.extend(sublist)
    else:
        X_processed.append(sublist)
X_processed = np.array(X_processed)

IndexError: list index out of range

In [200]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
num_classes = len(label_encoder.classes_)

In [206]:
X_train = np.reshape(X_processed, (X_processed.shape[0], 1, X_processed.shape[1]))

In [214]:
model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(padded_arr, y_encoded, epochs=10, batch_size=32,)


ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {'(<class \'list\'> containing values of types {"<class \'int\'>"})', '(<class \'list\'> containing values of types {"<class \'numpy.ndarray\'>"})', '(<class \'list\'> containing values of types {"<class \'numpy.ndarray\'>", "<class \'int\'>"})'}), <class 'numpy.ndarray'>

In [213]:
np.array(padded_arr).shape

C:\Users\Sam\AppData\Local\Temp\ipykernel_508\1273227418.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(padded_arr).shape


(1059, 372)

In [228]:
t = np.load("X.npy", allow_pickle=True)

In [229]:
t = np.array(t)

In [230]:
t.shape

(1071, 618264)

In [231]:
t[34][-1]

0.0

In [16]:
import numpy as np

In [17]:
X = np.load("X.npy")

In [22]:
X

array([[ 0.45600423,  0.2333079 , -0.506378  , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.47096324,  0.23414364, -0.81818223, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.46856007,  0.21675549, -0.70816606, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.49104264,  0.27044585, -0.60848659, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.48748872,  0.28047171, -0.56018519, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.51354039,  0.24837087, -0.83206671, ...,  0.        ,
         0.        ,  0.        ]])

In [19]:
X[0][: 250]

array([ 4.56004232e-01,  2.33307898e-01, -5.06377995e-01,  9.99777853e-01,
        4.75985676e-01,  1.90068424e-01, -4.78294104e-01,  9.99634504e-01,
        4.89453167e-01,  1.89619482e-01, -4.78280246e-01,  9.99605238e-01,
        5.00471950e-01,  1.89701259e-01, -4.78520215e-01,  9.99657869e-01,
        4.34933573e-01,  1.94741786e-01, -4.77371871e-01,  9.99725521e-01,
        4.23198521e-01,  1.97234690e-01, -4.77218747e-01,  9.99715626e-01,
        4.14740801e-01,  2.00044155e-01, -4.77521986e-01,  9.99781072e-01,
        5.21495759e-01,  2.15462804e-01, -2.81138867e-01,  9.99603331e-01,
        4.07695651e-01,  2.25192621e-01, -2.63448089e-01,  9.99781907e-01,
        4.89424437e-01,  2.78147221e-01, -4.31510329e-01,  9.99895573e-01,
        4.39041227e-01,  2.79931903e-01, -4.26113307e-01,  9.99925017e-01,
        6.26905918e-01,  4.99994755e-01, -1.98663011e-01,  9.99234319e-01,
        3.38460505e-01,  5.08986771e-01, -1.92352444e-01,  9.99715507e-01,
        6.60757363e-01,  

In [12]:
new_x = X.reshape(X.shape[0], 372, 1662)

In [2]:
new_y = np.load("Y.npy", allow_pickle=True)

NameError: name 'np' is not defined

In [14]:
(new_x).shape

(1071, 372, 1662)

In [15]:
# new_x = new_x[:200]

In [16]:
new_x.shape

(1071, 372, 1662)

In [17]:
len(new_x[0][-2])

1662

In [18]:
# new_y = Y[:200]

In [19]:
new_y.shape

(1071,)

In [20]:
from sklearn.preprocessing import LabelEncoder

code = np.array(new_y)

label_encoder = LabelEncoder()
vec = label_encoder.fit_transform(code)

In [21]:
from keras.utils import to_categorical

y = to_categorical(vec)

In [22]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, BatchNormalization
# from tensorflow.keras.callbacks import TensorBoard

# model = Sequential()
# model.add(LSTM(64, activation='relu', input_shape=(372,1662)))
# model.add(BatchNormalization())
# # model.add(LSTM(128, return_sequences=True, activation='relu'))
# # model.add(LSTM(64, return_sequences=False, activation='relu'))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(189, activation='softmax'))

In [33]:
import tensorflow as tf
from tensorflow.keras import layers, optimizers

# include early stopping and reducelr
def get_callbacks():
    return [
            tf.keras.callbacks.EarlyStopping(
            # monitor="val_accuracy",
            patience = 10,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            # monitor = "val_accuracy",
            factor = 0.5,
            patience = 3
        ),
    ]

# a single dense block followed by a normalization block and relu activation
def dense_block(units, name):
    fc = layers.Dense(units)
    norm = layers.LayerNormalization()
    act = layers.Activation("relu")
    drop = layers.Dropout(0.5)
    return lambda x: drop(act(norm(fc(x))))

# the lstm block with the final dense block for the classification
def classifier(lstm_units):
    lstm = layers.LSTM(lstm_units)
    out = layers.Dense(878, activation="softmax")
    return lambda x: out(lstm(x))
# choose the number of nodes per layer
encoder_units = [512, 256] # tune this
lstm_units = 250 # tune this

#define the inputs (ragged batches of time series of landmark coordinates)
inputs = tf.keras.Input(shape=((372,1662)), ragged=True)

# dense encoder model
x = inputs
for i, n in enumerate(encoder_units):
    x = dense_block(n, f"encoder_{i}")(x)

# classifier model
out = classifier(lstm_units)(x)

model = tf.keras.Model(inputs=inputs, outputs=out)
model.summary()


Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 372, 1662)]       0         
                                                                 
 dense_14 (Dense)            (None, 372, 512)          851456    
                                                                 
 layer_normalization_10 (Lay  (None, 372, 512)         1024      
 erNormalization)                                                
                                                                 
 activation_10 (Activation)  (None, 372, 512)          0         
                                                                 
 dropout_10 (Dropout)        (None, 372, 512)          0         
                                                                 
 dense_15 (Dense)            (None, 372, 256)          131328    
                                                           

In [35]:
# optimizer = optimizers.RMSprop(learning_rate = 0.000001)

model.compile(optimizer="Adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"])
# fit the model with 100 epochs iteration
model.fit(new_x,
          y,
        #   callbacks = get_callbacks(),
          epochs = 500)

Epoch 1/500
34/34 [==============================] - 149s 4s/step - loss: 6.7691 - accuracy: 0.0056
Epoch 2/500
34/34 [==============================] - 161s 5s/step - loss: 6.7334 - accuracy: 0.0056
Epoch 3/500
34/34 [==============================] - 148s 4s/step - loss: 6.7333 - accuracy: 0.0056
Epoch 4/500
34/34 [==============================] - 141s 4s/step - loss: 6.7306 - accuracy: 0.0056
Epoch 5/500
34/34 [==============================] - 137s 4s/step - loss: 6.7280 - accuracy: 0.0056
Epoch 6/500
34/34 [==============================] - 129s 4s/step - loss: 6.7289 - accuracy: 0.0065
Epoch 7/500
34/34 [==============================] - 144s 4s/step - loss: 6.7272 - accuracy: 0.0056
Epoch 8/500
34/34 [==============================] - 147s 4s/step - loss: 6.7274 - accuracy: 0.0065
Epoch 9/500
34/34 [==============================] - 161s 5s/step - loss: 6.7287 - accuracy: 0.0065
Epoch 10/500
34/34 [==============================] - 170s 5s/step - loss: 6.7286 - accuracy: 0.0056

KeyboardInterrupt: 

In [18]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [55]:
# model.fit(new_x, y, epochs=10)

Epoch 1/10
7/7 [==============================] - 17s 1s/step - loss: 5.2433 - categorical_accuracy: 0.0000e+00
Epoch 2/10
7/7 [==============================] - 10s 1s/step - loss: 5.2421 - categorical_accuracy: 0.0050
Epoch 3/10
7/7 [==============================] - 10s 1s/step - loss: 5.2418 - categorical_accuracy: 0.0100
Epoch 4/10
7/7 [==============================] - 11s 1s/step - loss: 5.2416 - categorical_accuracy: 0.0100
Epoch 5/10
7/7 [==============================] - 11s 2s/step - loss: 5.2415 - categorical_accuracy: 0.0100
Epoch 6/10
7/7 [==============================] - 11s 2s/step - loss: 5.2414 - categorical_accuracy: 0.0100
Epoch 7/10
7/7 [==============================] - 11s 2s/step - loss: 5.2412 - categorical_accuracy: 0.0100
Epoch 8/10
7/7 [==============================] - 12s 2s/step - loss: 5.2411 - categorical_accuracy: 0.0100
Epoch 9/10
7/7 [==============================] - 13s 2s/step - loss: 5.2410 - categorical_accuracy: 0.0050
Epoch 10/10
7/7 [=======